# Classification Models in sklearn

Building classifier models to predict house prices in King County, USA

## Preliminaries

In [1]:
# To auto-reload modules in jupyter notebook (so that changes in files *.py doesn't require manual reloading):
# https://stackoverflow.com/questions/5364050/reloading-submodules-in-ipython
%load_ext autoreload
%autoreload 2

Import commonly used libraries and magic command for inline plotting

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


In [3]:
%matplotlib inline

## Read in raw data

In [4]:
housing_df = pd.read_csv("./data/kc_house_data_classification.csv")

In [5]:
housing_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21613 entries, 0 to 21612
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   bedrooms       21613 non-null  int64  
 1   bathrooms      21613 non-null  float64
 2   sqft_living    21613 non-null  int64  
 3   sqft_lot       21613 non-null  int64  
 4   floors         21613 non-null  float64
 5   waterfront     21613 non-null  int64  
 6   view           21613 non-null  int64  
 7   condition      21613 non-null  int64  
 8   grade          21613 non-null  int64  
 9   sqft_above     21613 non-null  int64  
 10  sqft_basement  21613 non-null  int64  
 11  yr_built       21613 non-null  int64  
 12  yr_renovated   21613 non-null  int64  
 13  zipcode        21613 non-null  int64  
 14  lat            21613 non-null  float64
 15  long           21613 non-null  float64
 16  sqft_living15  21613 non-null  int64  
 17  sqft_lot15     21613 non-null  int64  
 18  price_

## Automated EDA

Introducing pandas-profiling and Sweetviz to create interactive HTML based reports. (NB: The code cells for the installation and use of pandas-profiling and Sweetviz were executed in the classic jupyter notebook)

In [ ]:
# Run cell to install latest version of pandas profiling

# pip install https://github.com/ydataai/pandas-profiling/archive/master.zip

In [ ]:
# from pandas_profiling import ProfileReport

In [ ]:
# profile = ProfileReport(housing_df, title="Pandas Profiling Report")

In [ ]:
# profile.to_file("output/pandas_profiling_report.html")

In [ ]:
# pip install sweetviz

In [ ]:
# import sweetviz

In [ ]:
# report = sweetviz.analyze(housing_df)

In [ ]:
# report.show_html("output/sweetviz_report.html")

The pandas_profiling_report.html and sweetviz_report.html reports have been saved to the output folder. However, to view them in this notebook, run the next 2 code cells.

In [ ]:
# Display pandas_profiling_report.html in this jupyter notebook

from IPython.display import IFrame
IFrame(src="output/pandas_profiling_report.html", width=900, height=600)

In [ ]:
#Display sweetviz_report.html in this jupyter notebook 

IFrame(src="output/sweetviz_report.html", width=900, height=600)